In [ ]:
import numpy as np
import pandas as pd
import re
import gc

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from gensim.models import Word2Vec, KeyedVectors
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
%matplotlib inline

from tqdm import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/final main.csv")
df.shape

(66397, 2)

In [ ]:
df.shape

(66397, 2)

In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66397 entries, 0 to 66396
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_info  66397 non-null  object
 1   desc          66397 non-null  object
dtypes: object(2)
memory usage: 1.0+ MB


In [ ]:
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [1]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
model = KeyedVectors.load_word2vec_format('/root/input/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
df['words'] = df.desc.progress_apply(text_to_wordlist)


100%|██████████| 66397/66397 [00:05<00:00, 11991.19it/s]


In [ ]:
def text2vec(text):
    return np.mean([model[x] for x in text.split() if x in model.vocab], axis=0).reshape(1,-1)

In [ ]:
df['vectors'] = df.words.progress_apply(text2vec)


 26%|██▋       | 17511/66397 [00:02<00:06, 7985.60it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
100%|██████████| 66397/66397 [00:08<00:00, 7406.05it/s]


In [ ]:
df.head()

,product_info,desc,words,vectors
0,SEER SECRETS ACTIVE SILVER ION DEODORANT CREAM,1mg body lotion with shea butter vitamin e p....,1mg body lotion with shea butter vitamin e p 1...,"[[-0.039584126, 0.070044145, 0.05129129, 0.045..."
1,SEER SECRETS MILK BODY LOTION SPF 21,1mg body lotion with shea butter vitamin e p....,1mg body lotion with shea butter vitamin e p 1...,"[[-0.042305257, 0.066261634, 0.049816526, 0.04..."
2,VITRO NATURALS ALOE SHEA BODY BUTTER,1mg body lotion with shea butter vitamin e p....,1mg body lotion with shea butter vitamin e p 1...,"[[-0.042305257, 0.066261634, 0.049816526, 0.04..."
3,1MILE HEALTHCARE CLASSIC TONE STETHOSCOPE,1mile healthcare classic tone stethoscope ......,1mile healthcare classic tone stethoscope orde...,"[[-0.0073661804, 0.011094868, 0.008302689, 0.0..."
4,1MILE HEALTHCARE DOCTOR PLUS STETHOSCOPE,1mile healthcare doctor plus stethoscope 606…...,1mile healthcare doctor plus stethoscope 606 o...,"[[-0.05913925, -0.024748027, -0.003479004, 0.0..."


In [ ]:
df.to_pickle('/content/drive/My Drive/final main.pkl')

In [ ]:
X = np.concatenate(df['vectors'].values,axis=-1)

In [ ]:
X

array([[-0.03958413,  0.07004414,  0.05129129, ..., -0.15527344,
        -0.29492188, -0.15234375]])

In [ ]:
kmeans = KMeans(n_clusters=1000)
kmeans.fit(X)
df['cluster'] = kmeans.predict(X)

ValueError: ignored

In [ ]:
clusters = df.groupby('cluster')    

In [ ]:
for cluster in clusters.groups:
    f = open('cluster'+str(cluster)+ '.csv', 'w') # create csv file
    data = clusters.get_group(cluster)[['product_info','desc']] # get title and overview columns
    f.write(data.to_csv(index_label='id')) # set index to id
    f.close()

print("Cluster centroids: \n")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

for i in range(k):
    print("Cluster %d:" % i)
    for j in order_centroids[i, :10]: #print out 10 feature terms of each cluster
        print (' %s' % terms[j])
    print('------------')

In [ ]:
new_doc = ["sanitary pads"]
Y = text2vec(new_doc)
prediction = model.predict(Y)
print(prediction)

In [ ]:
result=pd.read_csv("cluster{}.csv".format(prediction[0]))

In [ ]:
for i  in result["desc"]:
  print(i)